In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/DLAA_2025/qwen2.5-0.5b-instruct-lora-output"
%ls
!pwd

Mounted at /content/drive
/content/drive/My Drive/DLAA_2025/qwen2.5-0.5b-instruct-lora-output
checkpoint-20/  evaluation_avant.json          evaluation_checkpoint-60.json
checkpoint-40/  evaluation_checkpoint-20.json  plotdiag/
checkpoint-60/  evaluation_checkpoint-40.json
/content/drive/My Drive/DLAA_2025/qwen2.5-0.5b-instruct-lora-output


In [2]:
import json
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import euclidean_distances
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
import sys

import numpy as np

# Add parent directory to sys.path
print(os.path.abspath)
sys.path.append(os.path.abspath(".."))
#from ts_dataset import ask_noimage

<function abspath at 0x79cf34c860c0>


In [3]:
# Create the directory if it does not exist
output_dir = 'plotdiag'
os.makedirs(output_dir, exist_ok=True)

In [4]:
# Load sentence transformer model
model_name = "paraphrase-MiniLM-L6-v2"
print("Loading model " + model_name)
model_st = SentenceTransformer(model_name)

Loading model paraphrase-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Load a model trained for contradiction detection (NLI)
nli_model_name = "roberta-large-mnli"
print("Loading NLI model " + nli_model_name)
nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name)

def compute_semantic_similarity(gold_output, generated_output):
    emb_generated = model_st.encode([generated_output], convert_to_tensor=True)
    emb_gold = model_st.encode([gold_output], convert_to_tensor=True)

    cosine_score = torch.nn.functional.cosine_similarity(emb_generated, emb_gold).cpu().item()
    euclidean_score = euclidean_distances(emb_generated.cpu(), emb_gold.cpu())[0][0]

    return cosine_score, euclidean_score

def detect_contradiction_nli(premise, hypothesis):
    inputs = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors='pt', truncation=True)
    with torch.no_grad():
        logits = nli_model(**inputs).logits
    probs = F.softmax(logits, dim=1)
    #labels = ['entailment', 'neutral', 'contradiction']
    labels = ['no', 'bof', 'ok']
    max_idx = torch.argmax(probs).item()
    return labels[max_idx], probs[0][max_idx].item()


# test the detection of contradiction
premise = "The cat is in the tree."
hypothesis = "The moggy is swimming."
label, score = detect_contradiction_nli(premise, hypothesis)
print(f"Label: {label}, Score: {score}")


Loading NLI model roberta-large-mnli


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Label: no, Score: 0.9918491840362549


In [6]:
file_name = 'evaluation_checkpoint-40.json'
#file_name = 'evaluation_avant.json'
# Load the specified JSON file
with open(file_name, 'r') as f:
    evaluation_apres = json.load(f)

In [7]:
# compute verified annotations from ground truth
def ask_truth(X):
    x = np.arange(len(X))
    coeffs = np.polyfit(x, X, deg=3)
    P = np.poly1d(coeffs)
    X_fit = P(x)
    # compute the l2 norm of the difference
    l2_norm = np.linalg.norm(X - X_fit)/np.sqrt(len(X))
    # plt.plot(x,X)
    # plt.plot(x,X_fit)
    # plt.show()
    Pp = P.deriv()
    Xp_fit = Pp(x)
    delta = Xp_fit[-1] - Xp_fit[0]
    # divmin = np.min(Xp_fit)
    # divmax = np.max(Xp_fit)
    # if divmin > 0:
    #     sentence =  "the time series presents an overall increasing trend"
    # elif divmax < 0:
    #     sentence = "the time series presents an overall decreasing trend"
    # else:
    #     sentence = "the time series presents no uniformly increasing or decreasing trend"
    # if delta > 5:
    #     sentence =  "the time series presents an overall increasing trend"
    # elif delta < -5:
    #     sentence = "the time series presents an overall decreasing trend"
    # else:
    #     sentence = "the time series presents no uniformly increasing or decreasing trend"
    # compute the average of the solution on the 20 first points
    average1 = np.mean(X[:20])
    # compute the average of the solution on the 20 last points
    average2 = np.mean(X[-20:])
    if average1 < average2 -3:
        sentence_trend = "the time series shows an overall increasing trend."
    elif average1 > average2 +3:
        sentence_trend = "the time series shows an overall decreasing trend."
    else:
        sentence_trend = "the time series shows no uniformly increasing or decreasing trend."

    print(f"L2 norm of the difference: {l2_norm}")
    #sentence_noise = self.data_json[i]["description"]["noise"]
    if l2_norm < 2:
        sentence_noise = "the noise intensity is low"
    elif l2_norm > 12:
        sentence_noise = "the noise intensity is high"
    else:
        sentence_noise = "the noise intensity is medium"
    # print(sentence_noise)
    # self.data_json[i]["truth_description"]["noise"] = sentence_noise
    # recherche de la localisation en t du maximum et du minimum
    pos_max = np.argmax(X)
    print('pos_max', pos_max)
    if pos_max < 32:
        sentence_extrema = "The maximum is reached around the beginning part of the time series"
    elif pos_max > 96:
        sentence_extrema = "The maximum is reached towards the end of the time series"
    else:
        sentence_extrema = "The maximum is reached around the middle of the time series"

    pos_min = np.argmin(X)
    print('pos_min', pos_min)
    if pos_min < 32:
        sentence_extrema += " and the minimum is reached around the beginning part of the time series."
    elif pos_min > 96:
        sentence_extrema += " and the minimum is reached towards the end of the time series."
    else:
        sentence_extrema += " and the minimum is reached around the middle of the time series."

    return (sentence_trend, sentence_noise, sentence_extrema)

In [8]:
import ast
import re
def plot_and_save(i):
    input_data = evaluation_apres[i]['input']
    series_str = input_data.split('Series: ')[1].strip()
    series_str = re.sub(r'\b0+(\d)', r'\1', series_str)
    series = ast.literal_eval(series_str)


    truth_trend, truth_noise, truth_extrema = ask_truth(series)
    print("truth_trend: ", truth_trend)
    print("truth_noise: ", truth_noise)
    print("truth_extrema: ", truth_extrema)
    #series = json.loads(input_data.split('Series: ')[1])
    #exit()
    gold_output = evaluation_apres[i]['gold_output']
    # diagnostic_avant = evaluation_avant[i]['generated_output']
    # cute the string to the first 600 characters
    # diagnostic_avant = diagnostic_avant[:600]
    diagnostic_apres = evaluation_apres[i]['generated_output']
    print("diagnostic_apres: ", diagnostic_apres)
    # in this string keep the string that is between { }
    diagnostic_apres = diagnostic_apres[diagnostic_apres.find('{'):diagnostic_apres.rfind('}')+1]
    print("diagnostic_apres: ", diagnostic_apres)

    sentence_gold = []
    sentence_after = []

    # extract json object from the string before
    json_gold = json.loads(gold_output)
    sentence_gold.append(json_gold['trend'])
    sentence_gold.append(json_gold['noise'])
    sentence_gold.append(json_gold['extrema'])

    sentence_gold = [truth_trend, truth_noise, truth_extrema]

    try:
        json_after = json.loads(diagnostic_apres)
        print("json_after: ", json_after)
        sentence_after.append(json_after['trend'])
        sentence_after.append(json_after['noise'])
        sentence_after.append(json_after['extrema'])
        print("sentence_after: ", sentence_after)
    except (json.JSONDecodeError, KeyError):
        sentence_after.extend([""] * 3)

    print("sentence_after: ", sentence_after)


    # Compute similarity scores for each sentence
    print(f"Calculating similarity scores for entry {i}...")
    cosine_score_gold, euclidean_score_gold = [] , []
    cosine_score_apres, euclidean_score_apres = [] , []
    cass_label_gold, cass_score_gold = [] , []
    cass_label_apres, cass_score_apres = [] , []
    # for each sentence in the json object
    for iss in range(len(sentence_gold)):
        # Ensure inputs are strings
        after_text = str(sentence_after[iss]) if not isinstance(sentence_after[iss], str) else sentence_after[iss]
        gold_text = str(sentence_gold[iss]) if not isinstance(sentence_gold[iss], str) else sentence_gold[iss]

        a , b = compute_semantic_similarity(after_text, gold_text)
        print(after_text, gold_text, a,b)
        cosine_score_apres.append(a)
        euclidean_score_apres.append(b)
        a,b = detect_contradiction_nli(after_text, gold_text)
        cass_label_apres.append(a)
        cass_score_apres.append(b)

    # Concatenate the similarity scores and CASS labels properly
    for cosine, euclidean, cass_label, cass_score in zip(cosine_score_apres, euclidean_score_apres, cass_label_apres, cass_score_apres):
        diagnostic_apres += f"\n\nCosine Score: {cosine:.4f}\nEuclidean Distance: {euclidean:.4f}"
        diagnostic_apres += f"\n\n[CASS] Relation: {cass_label} (score: {cass_score:.4f})"


    fig, axs = plt.subplots(1+3, 1, figsize=(10, 12))
    axs[0].set_ylim(0, 99)
    axs[0].plot(series)
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Value')
    axs[0].set_title('Time Series')


    for iss in range(len(sentence_gold)):
        axs[1+iss].axis('off')
        disp = 'gold: '+str(sentence_gold[iss]) + ' \n' + 'after: ' + str(sentence_after[iss]) + ' \n' + 'cosine: ' + str(cosine_score_apres[iss]) + ' \n' + 'euclidean: ' + str(euclidean_score_apres[iss]) + ' \n' + 'cass: ' + cass_label_apres[iss] + ' \n' + 'cass score: ' + str(cass_score_apres[iss])
        color = 'red' if cass_label_apres[iss] == 'no' else 'orange' if cass_label_apres[iss] == 'bof' else 'green'
        axs[1+iss].text(0.1, 0.5, disp, fontsize=10, verticalalignment='center', wrap=True, color=color)

    # color_avant = 'red' if cass_label_gold == 'contradiction' else 'orange' if cass_label_gold == 'neutral' else 'blue'
    # color_apres = 'red' if cass_label_apres == 'contradiction' else 'orange' if cass_label_apres == 'neutral' else 'green'

    # axs[2].axis('off')
    # axs[2].text(0.1, 0.5, diagnostic_avant, fontsize=10, verticalalignment='center', wrap=True, color=color_avant)

    # axs[2].axis('off')
    # axs[2].text(0.1, 0.5, diagnostic_apres, fontsize=10, verticalalignment='center', wrap=True, color=color_apres)

    fig.suptitle(f'Case number {i}', fontsize=16)
    file_name = os.path.join(output_dir, f'case_{i}.png')
    print(f"Saving figure to {file_name}")
    fig.savefig(file_name)
    plt.close(fig)
    scores = [0 if ca == 'no' else 1 if ca == 'ok' else 0.5 for ca in cass_label_apres]
    return scores

In [9]:
mean_scores = [0.,0.,0.]
for i in range(len(evaluation_apres)):
  scores = plot_and_save(i)
  print(f"Scores for case {i}: {scores}")
  for j in range(3):
    mean_scores[j] += scores[j]
for j in range(3):
  mean_scores[j] /= len(evaluation_apres)
print(f"Mean scores: {mean_scores}")


L2 norm of the difference: 11.104642775646
pos_max 72
pos_min 1
truth_trend:  the time series shows an overall increasing trend.
truth_noise:  the noise intensity is medium
truth_extrema:  The maximum is reached around the middle of the time series and the minimum is reached around the beginning part of the time series.
diagnostic_apres:  {"trend": "The time series shows an overall increasing trend.", "noise": "The noise intensity is medium", "extrema": "The global maximum is located towards the end, while the global minimum is at the beginning."}
diagnostic_apres:  {"trend": "The time series shows an overall increasing trend.", "noise": "The noise intensity is medium", "extrema": "The global maximum is located towards the end, while the global minimum is at the beginning."}
json_after:  {'trend': 'The time series shows an overall increasing trend.', 'noise': 'The noise intensity is medium', 'extrema': 'The global maximum is located towards the end, while the global minimum is at the b